# Swinburne_project

This project is to understand community attitudes towards violence against women in Australia, provided by IT departments in Swinburne University of Technology.

Data are from the Twitter and Instagram.

Research scope is Melbourne metropolitan region.

Source code: https://github.com/Tiangewang0524/Swinburne_project


** There is the shortage on the raw data. Will be discussed in the end. 

## 0. Requisite package:

matplotlib

vaderSentiment

json

wordcloud

nltk

* If you are doing this for first time you need to use nltk.download('punkt') for word_tokenize to work and nltk.download('averaged_perceptron_tagger') for pos_tag to work. 


* Part-of-speech tags list:

  https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html



## 1. Data collect 

raw_data.sh

A shell script to collect Twitter and Instagram raw data from AURIN/tweet-infra: https://github.com/AURIN/tweet-infra

curl "http://45.113.232.90/couchdbro/instagram/_design/instagram/_view/summary" \
-G \
--data-urlencode 'start_key=["melbourne",2017,1,1]' \
--data-urlencode 'end_key=["melbourne",2018,3,12]' \
--data-urlencode 'reduce=false' \
--data-urlencode 'include_docs=true' \
--user "readonly:ween7ighai9gahR6" \
-o /tmp/raw_instagram_data.json

curl "http://45.113.232.90/couchdbro/twitter/_design/twitter/_view/summary" \
-G \
--data-urlencode 'start_key=["melbourne",2017,6,15]' \
--data-urlencode 'end_key=["melbourne",2018,5,4]' \
--data-urlencode 'reduce=false' \
--data-urlencode 'include_docs=true' \
--user "readonly:ween7ighai9gahR6" \
-o /tmp/raw_twitter_data.json

## 2. Functions 

In [5]:
import json
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# import matplotlib.ticker as mtick

ModuleNotFoundError: No module named 'wordcloud'

* convert:

To convert numbers to months.

In [6]:
# convert numbers to months
def convert(x):
    return{
        1:'Jan',
        2:'Feb',
        3:'Mar',
        4:'Apr',
        5:'May',
        6:'Jun',
        7:'Jul',
        8:'Aug',
        9:'Sep',
        10:'Oct',
        11:'Nov',
        12:'Dec'
    }.get(x)

* filtered:

Clean and filter Twitter/Instagram raw data with hashtag #metoo & #timesup and reserve 5 attributes: "tweet_id", "user_id", "user_name", "post_text" and "post_time"

In [7]:
# filter Twitter data
def twi_filtered(read_file, write_file):
    info = dict()
    write_file.write('{"rows":[\n')
    for each_row in read_file:
        try:
            # Remove the blank and comma in the end of each row.
            item = json.loads(each_row[:-2])
            # print(item)
            sub_post = item["doc"]["text"].split()
            if key_hashtag_1 in sub_post or key_hashtag_2 in sub_post:
                info["post_time"] = item["doc"]["created_at"]
                info["tweet_id"] = item["doc"]["id_str"]
                info["user_id"] = item["doc"]["user"]["id_str"]
                info["user_name"] = item["doc"]["user"]["name"]
                info["post_text"] = item["doc"]["text"]
                each = json.dumps(info)
                write_file.write(each + ',' + '\n')
                write_file.flush()
                print("User " + info["user_name"] + "'s filtered task done!")
        except:
            continue
    write_file.seek(write_file.tell() - 2)
    write_file.write(']}')


# filter Instagram data
def ins_filtered(read_file, write_file):
    list_a = []
    info = dict()
    write_file.write('{"rows":[\n')
    for each_row in read_file:
        try:
            # Remove the blank and comma in the end of each row.
            item = json.loads(each_row[:-2])
            # Removed '#' from hashtag
            if ins_tag_1 in item["doc"]["tags"] or ins_tag_2 in item["doc"]["tags"]:
                # created_time is a string of numbers which are hard to understand
                # and no document to interpret what they represent.
                # info["post_time"] = item["doc"]["created_time"]
                list_a.append(item["key"][1])
                list_a.append(convert(item["key"][2]))
                list_a.append(item["key"][3])
                info["post_time"] = list_a
                list_a = []
                info["ins_post_id"] = item["doc"]["caption"]["id"]
                info["user_id"] = item["doc"]["user"]["id"]
                info["user_name"] = item["doc"]["user"]["username"]
                info["post_text"] = item["doc"]["caption"]["text"]
                each = json.dumps(info)
                print(each)
                write_file.write(each + ',' + '\n')
                write_file.flush()
                print("User " + info["user_name"] + "'s filtered task done!")
        except:
            continue
    write_file.seek(write_file.tell() - 2)
    write_file.write(']}')

* sentiment:

Sentiment analysis on Twitter/Instagram filtered data based on vaderSentiment package. To evaluate the attitude (positive or negative) of each Twitter/Instagram post.

In [8]:
# twitter sentiment analysis
def twi_sentiment(read_file, write_file):
    month_attitude = dict()
    sid = SentimentIntensityAnalyzer()
    info = dict()
    write_file.write('{"rows":[\n')
    for each_row in read_file:
        try:
            # Remove the blank and comma in the end of each row.
            item = json.loads(each_row[:-2])
            post_time = item["post_time"].split()
            date = post_time[5] + "," + post_time[1]
            if date not in month_attitude.keys():
                month_attitude[date] = dict()
                month_attitude[date]["count"] = 0
                month_attitude[date]["pos_count"] = 0
                month_attitude[date]["neg_count"] = 0
            month_attitude[date]["count"] += 1
            sentence = item["post_text"]

            # evaluate the sentiment for each tweet and save as a new file for further analysis.
            info = dict(item)

            # 1. Positive score – Negative score > 0 => Positive sentiment.
            # 2. Positive score – Negative score < 0 => Negative sentiment.
            if sid.polarity_scores(sentence)['neg'] - sid.polarity_scores(sentence)['pos'] > 0:
                month_attitude[date]["neg_count"] += 1
                info["attitude"] = "negative"
            elif sid.polarity_scores(sentence)['neg'] - sid.polarity_scores(sentence)['pos'] < 0:
                month_attitude[date]["pos_count"] += 1
                info["attitude"] = "positive"

            each = json.dumps(info)
            write_file.write(each + ',' + '\n')
            write_file.flush()
            print("User " + info["user_name"] + "'s attitude task done!")
        except:
            continue
    write_file.seek(write_file.tell() - 2)
    write_file.write(']}')
    return month_attitude


# instagram sentiment analysis
def ins_sentiment(read_file, write_file):
    month_attitude = dict()
    sid = SentimentIntensityAnalyzer()
    info = dict()
    write_file.write('{"rows":[\n')
    for each_row in read_file:
        try:
            # Remove the blank and comma in the end of each row.
            item = json.loads(each_row[:-2])
            date = str(item["post_time"][0]) + "," + str(item["post_time"][1])
            if date not in month_attitude.keys():
                month_attitude[date] = dict()
                month_attitude[date]["count"] = 0
                month_attitude[date]["pos_count"] = 0
                month_attitude[date]["neg_count"] = 0
            month_attitude[date]["count"] += 1
            sentence = item["post_text"]

            # evaluate the sentiment for each tweet and save as a new file for further analysis.
            info = dict(item)

            # 1. Positive score – Negative score > 0 => Positive sentiment.
            # 2. Positive score – Negative score < 0 => Negative sentiment.
            if sid.polarity_scores(sentence)['neg'] - sid.polarity_scores(sentence)['pos'] > 0:
                month_attitude[date]["neg_count"] += 1
                info["attitude"] = "negative"
            elif sid.polarity_scores(sentence)['neg'] - sid.polarity_scores(sentence)['pos'] < 0:
                month_attitude[date]["pos_count"] += 1
                info["attitude"] = "positive"

            each = json.dumps(info)
            write_file.write(each + ',' + '\n')
            write_file.flush()
            print("User " + info["user_name"] + "'s attitude task done!")
        except:
            continue
    write_file.seek(write_file.tell() - 2)
    write_file.write(']}')
    return month_attitude

* draw_sentiment:

Draw diagrams based on the result above via matplotlib package.

In [9]:
# draw the sentiment analysis
def draw_sentiment(attitude):
    # y axis related
    y_axis_pos = list()
    y_axis_neg = list()
    y_axis_difference = list()
    for each_data in attitude.values():
        y_axis_pos.append(int(-(each_data['pos_count'])))
        y_axis_neg.append(int(each_data['neg_count']))
        y_axis_difference.append(int(each_data['neg_count']) - int(each_data['pos_count']))

    # X axis related
    x_axis = list()
    for month in attitude.keys():
        x_axis.append(month)
    x_length = [i for i in range(len(x_axis))]

    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.plot(x_length, y_axis_difference, 'or-', label='difference')
    ax1.set_title('The comparison between positive and negative comments\n', fontsize=12, color='r')
    for i, (_x, _y) in enumerate(zip(x_length, y_axis_difference)):
        plt.text(_x, _y, y_axis_difference[i], color='black', fontsize=10)

    # left legend
    ax1.legend(loc=1)
    ax1.set_ylim([-500, 1000])
    ax1.set_ylabel('difference')
    ax2 = ax1.twinx()
    plt.bar(x_length, y_axis_pos, alpha=0.3, color='blue', label='pos_count')
    plt.bar(x_length, y_axis_neg, alpha=0.3, color='red', label='neg_count')

    # right legend
    ax2.legend(loc=2)
    ax2.set_ylim([-500, 1000])
    ax2.set_ylabel('number of count')

    for i, (_x, _y) in enumerate(zip(x_length, y_axis_pos)):
        plt.text(_x, _y - 50, y_axis_pos[i], color='maroon', fontsize=8, ha='center', va='bottom')
    for i, (_x, _y) in enumerate(zip(x_length, y_axis_neg)):
        plt.text(_x, _y + 5, y_axis_neg[i], color='maroon', fontsize=8, ha='center', va='bottom')

    plt.xticks(x_length, x_axis)
    plt.show()


Sentiment results are shown below:


<img style="float: middle;" src="https://github.com/Tiangewang0524/Swinburne_project/blob/master/dataset/twi_sentiment.png?raw=true" width="100%">



<img style="float: middle;" src="https://github.com/Tiangewang0524/Swinburne_project/blob/master/dataset/ins_sentiment.png?raw=true" width="100%">

* find_trollers:

Find trollers (who always have negative comments on women).

In [10]:
# find trollers
def find_troller(read_file, write_file):
    # count positive and negative comments for all users
    user_list = dict()
    nega_user = dict()
    troller_user = dict()
    write_file.write('{"Users":[\n')
    for each_row in read_file:
        try:
            # Remove the blank and comma in the end of each row.
            item = json.loads(each_row[:-2])
            user_id = item["user_id"]
            if user_id not in user_list.keys():
                user_list[user_id] = dict()
                user_list[user_id]["count"] = 0
                user_list[user_id]["pos_count"] = 0
                user_list[user_id]["neg_count"] = 0
            user_list[user_id]["count"] += 1
            if item["attitude"] == "negative":
                user_list[user_id]["neg_count"] += 1
            elif item["attitude"] == "positive":
                user_list[user_id]["pos_count"] += 1
        except:
            continue

    # for better sort, count negative comments for users and save as a dict.
    for each_user in user_list.keys():
        nega_user[each_user] = user_list[each_user]["neg_count"]

    # Select top 10 trollers who posted most negative commments on women.
    sort_user = sorted(nega_user.items(), key=lambda item:item[1], reverse=True)
    # count for 10 times
    count_a = 0
    read_file.seek(0)
    for each in sort_user:
        # each[0] is the user_id, each[1] is the number of negative comments.
        troller_user["user_id"] = each[0]
        for each_row in read_file:
            try:
                # Remove the blank and comma in the end of each row.
                item = json.loads(each_row[:-2])
                if each[0] == item["user_id"]:
                    troller_user["user_name"] = item["user_name"]
                    read_file.seek(0)
                    break
            except:
                continue
        troller_user["negative_comments"] = each[1]
        each = json.dumps(troller_user)
        write_file.write(each + ',' + '\n')
        write_file.flush()
        print("Troller " + troller_user["user_name"] + " has been found!")
        troller_user.clear()

        count_a += 1
        if count_a == 10:
            break

    write_file.seek(write_file.tell() - 2)
    write_file.write(']}')


* word_identify:

Collect all the tweet and instagram posts, split them and identify the part-of-speech of the words. Then store as a txt file.

In [11]:
# Collect all the tweet and instagram posts, split them and identify the part-of-speech of the words.
# Then store as a txt file.
def word_identify(read_file, write_file):
    text = list()
    for each_row in read_file:
        try:
            # Remove the blank and comma in the end of each row.
            item = json.loads(each_row[:-2])
            # Remove the identical retweets
            if item["post_text"] not in text:
                text.append(item["post_text"])
        except:
            continue
    str_text = " ".join(text)

    # split the str_text
    tokens = nltk.word_tokenize(str_text)
    # Add part-of-speech tag for each word
    tagged = nltk.pos_tag(tokens)
    text = list()
    for each in tagged:
        # each format : (word, 'tag')
        # Reserve nouns, verbs and adjectives, remove RT, @, https, SSS and URL。
        if each[1][0] == 'N' or each[1][0] == 'J' or each[1] == 'VB':
            if each[0] != '@' and each[0] != 'metoo' and each[0] != 'timesup' and each[0][0] != '/':
                if each[0] != 'RT' and each[0] != 'https' and each[0] != 'SSS':
                    text.append(each[0])

    str_text = " ".join(text)
    write_file.write(str_text)
    print("Word_identify task done!")
    return str_text


* word_cloud:

Generate word cloud for Twitter/Instagram filtered data.

In [12]:
# word cloud display
def word_cloud(str_text):
    wordcloud = WordCloud(background_color="white", width=1000, height=860, margin=2).generate(str_text)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

    file_name = input("input file name:")
    wordcloud.to_file(file_name)
    print("Word_cloud task done!")

WordCloud results are shown below:


<img style="float: middle;" src="https://github.com/Tiangewang0524/Swinburne_project/blob/master/dataset/twi_word_cloud.png?raw=true" width="60%">


<img style="float: middle;" src="https://github.com/Tiangewang0524/Swinburne_project/blob/master/dataset/ins_word_cloud.png?raw=true" width="60%">

## 3. Reflection 

Since the Twitter API has 7-day limit which means no tweets will be found for a date older than one week. So I had to use Prof. Richard's cloud to obtain raw Twitter data. Unfortunately, the data of Melbourne tweets stored in the cloud are from 15th JUN, 2017 to 4th MAY, 2018. Thus, I only gained 140 GB raw Twitter data.

Similarly, the Instagram API also has strict requirement when signing up for the 'access_token'. It asked me to demonstrate some reasons of using their API and record a video to display what I'm researching for (Very strange). So I also use Richard's cloud but only got 12 GB raw Instagram data from 1ST JAN, 2017 to 12th MAR, 2018. 

After testing, these functions doesn't show any bugs and fully satify with the requirements of tasks. However, as the crawler step is not created by myself and I have no way of knowing the detail, I can't figure out why the numbers of positive/negative counts in some months are too small. 

I guess:

1. The data for some days in some months in Richard's cloud are missing. 

2. Richard's cloud will not record those tweets/Instagram posts without the coordinates although this kind of users who posted them in Melbourne region. 

Functions can be use for any kinds and any period of Twitter/Instagram data. Once we have the chance to get more  abundant data, we could gain more accurate result.